In [1]:
import mne
mne.viz.set_browser_backend("matplotlib")
# mne.viz.set_3d_backend("notebook")
from matplotlib import pyplot as plt
%config InlineBackend.figure_format='retina'
import numpy as np
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids
from itertools import product
from tqdm import tqdm
import os

Using matplotlib as 2D backend.


In [2]:
data_dir = '/mnt/sphere/projects/image_decoding_from_brain/THINGS-MEG-raw/download'
subject_file = data_dir + '/participants.tsv'
subjects = pd.read_csv(subject_file, sep="\t")
def get_subject_id(x):
    return x.split("-")[1]  # noqa
subjects = subjects.participant_id.apply(get_subject_id).values
subject = subjects[0]
tasks = ['main']
sessions = ['{:02d}'.format(x) for x in range(1,13)]  # 2 recording sessions
runs = ['{:02d}'.format(x) for x in range(1,11)]

In [3]:
print('Loading CLIP-vision...')

Loading CLIP-vision...


In [4]:
clip_vision = np.load('/mnt/sphere/projects/image_decoding_from_brain/data/extracted_features/things_clipvision.npy', mmap_mode='r')

In [5]:
print('CLIP-vision shape: ', clip_vision.shape)

CLIP-vision shape:  (26107, 257, 768)


In [6]:
for session, task, run in tqdm(product(sessions, tasks, runs), total=len(sessions)*len(tasks)*len(runs), desc=f'Processing subject {subject}'):
    bids_path = BIDSPath(
        subject=subject,
        session=session,
        task=task,
        root=data_dir,
        run = run,
        datatype="meg",
    )
    raw = read_raw_bids(bids_path, verbose='ERROR')
    # break
    events = list()
    for annot in raw.annotations:
        # event = eval(annot.pop("description"))
        event = dict()
        event['kind'] = annot['description'].split('/')[0]
        if annot['description'].split('/')[0] != 'catch':
            event['image_id'] = annot['description'].split('/')[1]
        else:
            event['image_id'] = '-1' # catch trials, maybe we should change this
        event['start'] = annot['onset']
        event['duration'] = annot['duration']
        events.append(event)
    events_df = pd.DataFrame(events)
    # train_events_df = events_df[events_df['kind'] == 'exp']
    # test_events_df = events_df[events_df['kind'] == 'test']
    # train_image_ids = pd.to_numeric(train_events_df['image_id']).astype(int).to_numpy()
    # test_image_ids = pd.to_numeric(test_events_df['image_id']).astype(int).to_numpy()
    # if train_clip_vision is None:
    #     train_clip_vision = clip_vision[train_image_ids]
    #     test_clip_vision = clip_vision[test_image_ids]
    # else:
    #     train_clip_vision = np.concatenate((train_clip_vision, clip_vision[train_image_ids]), axis=0)
    #     test_clip_vision = np.concatenate((test_clip_vision, clip_vision[test_image_ids]), axis=0)
    if 0 in pd.to_numeric(events_df['image_id']).astype(int).to_numpy():
        print('image_id 0 is found in events_df')
    if 26106 in pd.to_numeric(events_df['image_id']).astype(int).to_numpy():
        print('image_id 26106 is found in events_df')

Processing subject BIGMEG1:  29%|██▉       | 35/120 [00:28<00:53,  1.58it/s]

image_id 26106 is found in events_df


Processing subject BIGMEG1: 100%|██████████| 120/120 [01:31<00:00,  1.31it/s]


In [ ]:
# save_dir = 'cache/extracted_embeddings/' + subject + '/'
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# np.save(save_dir + f'train_clipvision_sub-{subject}.npy', train_clip_vision)
# np.save(save_dir + f'test_clipvision_sub-{subject}.npy', test_clip_vision)

In [6]:
events = list()
for annot in raw.annotations:
    # event = eval(annot.pop("description"))
    event = dict()
    event['kind'] = annot['description'].split('/')[0]
    if annot['description'].split('/')[0] != 'catch':
        event['image_id'] = annot['description'].split('/')[1]
    else:
        event['image_id'] = '-1' # catch trials, maybe we should change this
    event['start'] = annot['onset']
    event['duration'] = annot['duration']
    events.append(event)

In [7]:
events_df = pd.DataFrame(events)
train_events_df = events_df[events_df['kind'] == 'exp']
test_events_df = events_df[events_df['kind'] == 'test']
train_image_ids = pd.to_numeric(train_events_df['image_id']).astype(int).to_numpy()
test_image_ids = pd.to_numeric(test_events_df['image_id']).astype(int).to_numpy()
image_ids = pd.to_numeric(events_df['image_id']).astype(int).to_numpy()
# events_df

In [23]:
clip_vision.shape

(26107, 257, 768)

In [8]:
clip_vision[image_ids].shape

(226, 257, 768)

In [25]:
abc = np.ones((26107,2,1))

In [26]:
abc[image_ids].shape

(226, 2, 1)

In [17]:
0 in pd.to_numeric(events_df['image_id']).astype(int).to_numpy()

False

In [12]:
paths_df = pd.read_csv('/mnt/sphere/projects/image_decoding_from_brain/THINGS-images/Metadata/Image-specific/image_paths.csv', header=None)
paths_df

,0
0,images/aardvark/aardvark_01b.jpg
1,images/aardvark/aardvark_02s.jpg
2,images/aardvark/aardvark_03s.jpg
3,images/aardvark/aardvark_04s.jpg
4,images/aardvark/aardvark_05s.jpg
...,...
26102,images/zucchini/zucchini_09s.jpg
26103,images/zucchini/zucchini_10s.jpg
26104,images/zucchini/zucchini_11s.jpg
26105,images/zucchini/zucchini_12s.jpg
